In [1]:
import numpy as np 
import pandas as pd
import os,re

In [42]:
import numpy as np 
import pandas as pd
import os,re
import nimfa
from collections import defaultdict

class probiotics_signature :

    def __init__(self,metadata,abundance_matrix) :
        """
        Init a probiotics signature object.

        Args:
            metadata (pd.Dataframe): clinical information or demographic data of patients / participants. row is sample, column is condition.
            abundance_matrix (pd.Dataframe): patient abundance matrix. row is species, column is sample.
        """        
        self.sample_metadata = None
        self.abundance_matrix = None

    def species_abundance_prevelence_scatterplot(self,input_matrix,output_path,format='pdf') :
        """
        Scatterplot of target species abundance / prevalence

        Args:
            input_matrix (pd.DataFrame): Target abundance matrix. row is species, column is sample.
            output_path (str): Output path of scatterplot.
            format (str, optional): Output figure format. Defaults to 'pdf'.
        """               
        mean_abundance = self.abundance_matrix.mean(axis=1)
        prevalence = self.abundance_matrix.apply(lambda x : (sum(x > 0)/len(x))*100,axis=1)
        plot_df = pd.DataFrame({'Prevalence' : prevalence,
                                'Mean_abundance' : mean_abundance,
                                'Species' : list(self.abundance_matrix.index)})

        plt.figure(figsize=(7,5))
        sns.scatterplot(data=plot_df,x='Prevalence',y='Mean_abundance',hue='Species',palette="rainbow_r",s=100)
        plt.xlabel("Prevalence(%)")
        plt.ylabel("Mean relative abundance(%)")
        plt.legend(bbox_to_anchor=[1,1],ncol=2,title="Species")
        plt.savefig(output_path,format=format,dpi=300,bbox_inches='tight')

    def concat_species_into_subtype(self,input_matrix,reference_df,genus='Lactobacillus',species_colname='species',subtype_colname='phylogroup') :
        """
         Merge abundance from rate species into subtype. (ex : Lactobacillus genus)

        Args:
            input_matrix (pd.DataFrame): Target abundance matrix. row is species, column is sample. The format of input matrix index must be Lactobacillus_acidophilus.
            reference_df (pd.DataFrame): Table with subtype information. Please make sure the format of species column is identical to input matrix index.
            species_colname (str, optional): Colname of species in reference_df. Defaults to 'species'.
            subtype_colname (str, optional): Colname of subtype in reference_df. Defaults to 'phylogroup'.

        Returns:
            subtype_matrix (pd.DataFrame): Abundance matrix of each subtype.
            subtype_dict (dict) : Dict of subtype and its components. Key is subtype, value is list of species
        """         
        speceis2subtype = dict(zip([x.replace(' ','_') for x in reference_df['species'].values],subtype_reference[subtype_colname].values))
        subtype_dict = defaultdict(list) 
        # subtype_dict format like : {'Levilactobacillus' : ['Lactobacillus_acetotolerans','Lactobacillus_acidophilus']}
        for species in input_matrix.index :
            if species in speceis2subtype :
                subtype = speceis2subtype[species]
            else :
                subtype = 'no phylogroup'

            if subtype == 'no phylogroup' :
                subtype_dict[genus + '_others'].append(species)
            else :
                subtype_dict[subtype+'_subtype'].append(species)

        subtype_matrix = pd.DataFrame(np.zeros([len(subtype_dict),input_matrix.shape[1]]),
        index = list(subtype_dict.keys()),columns=input_matrix.columns)
        for subtype in subtype_dict :
            target_species = list(set(subtype_dict[subtype]).intersection(input_matrix.index))
            subtype_matrix.loc[subtype,:] = input_matrix.loc[target_species,:].sum()

        return subtype_matrix,subtype_dict   

    def evaluate_nmf_component(self,input_matrix,k_min=2,k_max=10) :
        """
        Evaluate optimal k component for NMF decomposition processing.

        Args:
            input_matrix (numpy.ndarray) : The original matrix for NNF (V).
            k_min (int, optional): The minimum component number. Defaults to 2.
            k_max (int, optional): The maximum component number. Defaults to 10.
        """        
        nmf = nimfa.Nmf(input_matrix,rank=max_k, max_iter=200,n_run=10)
        nmf_fit = nmf()
        #evaluation
        rank_list = list(range(min_k,max_k+1))
        evalation = nmf.estimate_rank(rank_range=[x for x in rank_list])
        #output the estimation result
        measurements = ['rss','evar','dispersion','cophenetic','kl']
        measurement_table = pd.DataFrame({'Rank' : rank_list})
        for m in measuments :
            measurement_table[m] = [evalation[x][m] for x in rank_list]
        return measurement_table

    def plot_nmf_rank(self,measurement_table,
                    measurement_1 = 'cophenetic',
                    measurement_2 = 'rss',
                    fig_output_path='nmf_evaluation.pdf',
                    fig_format='pdf') :
        """
        Visualation of NMF rank evaluation.

        Args:
            measurement_table (pd.DataFrame): The measurement result, including Rank, rss, evar, dispersion, cophenetic, kl.
            measurement_1 (str, optional): The first measurement of NMF rank. Defaults to 'cophenetic'.
            measurement_2 (str, optional): The second measurement of NMF rank. Defaults to 'rss'.
            fig_output_path (str, optional): The figure output path. Defaults to 'nmf_evaluation.pdf'.
            fig_format (str, optional): The figure output format. Defaults to 'pdf'.
        """  
        if measurement_1 not in measurement_table.columns :
            print("Please confirm %s in measurement table !" % measurement_1)
            return 
        if measurement_2 not in measurement_table.columns :
            print("Please confirm %s in measurement table !" % measurement_2)
            return

        plt.figure(figsize=(8,5))
        g = sns.lineplot(data=measurement_table,x ='Rank',y=measurement_1, color="orange",marker='o')
        sns.lineplot(data=measurement_table,x = 'Rank',y=measurement_2, ax=g.axes.twinx(),marker='o')
        g.legend(handles=[Line2D([], [], marker='o', color="orange", label=measurement_1), 
                        Line2D([], [], marker='o', label=measurement_2)])
        plt.savefig(fig_output_path,dpi=300,format=fig_format)

    def finger_print_proportion(x,w,h):
        """
        Calculate the contribution of each NMF decompose component.
        Args:
            x (np.array): The original matrix.
            w (np.array): The weight matrix of NMF decomposition.
            h (np.array): The coefficient matrix of NMF decomposition

        Returns:
            proportion_matrix (np.array): The contribution of each component.
        """        
        n_finger_print = w.shape[1]
        n_sample = w.shape[0]
        proportion_matrix = np.zeros([n_sample,n_finger_print])

        for i in range(n_sample) :
            total = sum(x[i,:])
            accum = 0
            if total == 0 :
                pass
            else :
                for j in range(n_finger_print) :
                    ab = sum(np.dot(w[i,j],h[j,:]))# type: ignore                
                    proportion_matrix[i,j] = ab / total
                    accum += (ab / total)
            #proportion_matrix[i,-1] = 1 - accum # not calculate residual

        return proportion_matrix

    def sklearn_decompose_probiotics_signature(self,input_matrix,k) :
        """
        Decompose k signatures from input abundance matrix.

        Args:
            input_matrix (pd.DataFrame) : The origin matrix (n_species * n_sample) to be decomposed. 
            k (int): Number of component expected to be decomposed.
        """        
        X = input_matrix.to_numpy()
        #sklearn.decomposition version
        nmf_model = NMF(n_components=k, init='random', random_state=0)
        nmf_model.fit(X)
        W = nmf_model.transform(X)
        H = nmf_model.components_

        finger_print_matrix = finger_print_proportion(X,W,H)
        index = [prefix + ' signature' + str(x) for x in range(1,k+1)]
        # format signature coefficient matrix (n_signature * n_species)
        sig_coefficient = pd.DataFrame(H,index=index,columns=input_matrix.index) 
        # format signature weight matrix (n_signature * n_sample)
        finger_print_df = pd.DataFrame(finger_print_matrix.T,index=index,columns=input_matrix.columns)
        finger_print_df[finger_print_df > 1] = 1
        finger_print_df[finger_print_df < 0] = 0

        return finger_print_df,sig_coefficient

    def nimfa_decompose_probiotics_signature(self,input_matrix,k) :
        """
        Decompose k signatures from input abundance matrix.

        Args:
            input_matrix (pd.DataFrame) : The origin matrix (n_species * n_sample) to be decomposed. 
            k (int): Number of component expected to be decomposed.
        """        
        X = input_matrix.to_numpy()
        #Nimfa version
        nmf_model = nimfa.Nmf(X,rank=k, max_iter=1000)
        nmf_fit = nmf_model()
        W = np.array(nmf_fit.basis())
        H = np.array(nmf_fit.coef())

        finger_print_matrix = finger_print_proportion(X,W,H)
        index = [prefix + ' signature' + str(x) for x in range(1,k+1)]
        # format signature coefficient matrix (n_signature * n_species)
        sig_coefficient = pd.DataFrame(H,index=index,columns=input_matrix.index) 
        # format signature weight matrix (n_signature * n_sample)
        finger_print_df = pd.DataFrame(finger_print_matrix.T,index=index,columns=input_matrix.columns)
        finger_print_df[finger_print_df > 1] = 1
        finger_print_df[finger_print_df < 0] = 0

        return finger_print_df,sig_coefficient

In [3]:
class probiotics_signature_visualizer :
    def __init__(self,signature_proportion_matrix,sig_coefficient_matrix,sample_distance_matrix,sample_metadata) :
        """
        Visualziation of probiotics signature result

        Args :
        signature_proportion_matrix (pd.Dataframe): n_signature * n_samples. The value of matrix represent the proportion of abundance contributed by signature in samples.
        sig_coefficient_matrix (pd.Dataframe): n_signature * n_species. The value of matrix represent the coefficient of NMF W matrix.
        metadata (pd.Dataframe):
        distance_matrix (pd.Dataframe): n_sample * n_sample distance matrix. The value inside the matrix is the distance between samples. 
            (ex : Unifrac / Bray-curtis distance)
        """        
    def plot_signature_relative_importance(self) :
        pass

    def plot_signature_relative_importance_heatmap(self) :
        pass

    def plot_pcoa_permanova_scatterplot(self) :
        pass

    def plot_mds_permanova_scatterplot(self) :
        pass

    def diversity_comparison_between_cluster(self,ordinate_method = 'PCoA') :
        pass

### initialization & load requirement

In [4]:
abundance_matrix = pd.read_csv("/home/bruce1996/data/MCI/metaphlan_result/metaphlan_format_table.txt",sep='\t',index_col=0)
metadata = pd.read_csv("/home/bruce1996/data/MCI/metadata/metadata_20221224.csv",sep=',',index_col=0)
distance_matrix = pd.read_csv("/home/bruce1996/data/MCI/metaphlan_result/latest_unweighted_unifrac_distance_matrix.txt",sep='\t',index_col=0)
subtype_reference = pd.read_csv("/home/bruce1996/data/MCI/subtyping/Lactobacillus_group.csv")
ps = probiotics_signature(abundance_matrix=abundance_matrix,metadata=metadata)

In [44]:
# lactobacillus subtype
lacto_m = abundance_matrix.loc[[bool(re.search("Lactobacillus",x)) for x in abundance_matrix.index],:]
lacto_m.index = [x[3:] for x in lacto_m.index]
subtype_matrix,subtype_dict = ps.concat_species_into_subtype(lacto_m,subtype_reference)

### Evaluation optimal k of NMF